In [1]:
import os
os.chdir("../")

In [2]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [3]:
resultDir = 'results'
problem = 'cauctions' # choices=['setcover', 'cauctions', 'facilities', 'indset']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [4]:
f1 = "cauctions_20210710-004536.csv"
f2 = "cauctions_20210710-095340.csv"
df1 = pd.read_csv(f"{resultDir}/{f1}")
df2 = pd.read_csv(f"{resultDir}/{f2}")

In [38]:
df = pd.concat([df1, df2], ignore_index=True)

In [39]:
# eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
# eval_file = eval_files[-1]
# eval_file = f"{resultDir}/cauctions_20210710-004536.csv"

# df = pd.read_csv(eval_file)
# df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = df.astype({'nlps': float, 'nnodes' : float})

df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()

In [40]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [41]:
dfgcnns_gmean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].mean()
dfgcnns_gmean

nnodes       stime
type   sampling_strategy                          
big    depthK             11029.200732  163.037758
       depthK2            11608.371907  174.765016
       uniform5           11567.794394  173.920877
medium depthK               793.676129   13.914498
       depthK2              836.439089   14.525380
       uniform5             846.944877   14.567127
small  depthK                59.625175    1.715663
       depthK2               62.636424    1.767164
       uniform5              61.296136    1.753091

In [42]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'uniform5')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [43]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   sampling_strategy                    
big    depthK             0.953440  0.937425
       depthK2            1.003508  1.004854
       uniform5           1.000000  1.000000
medium depthK             0.937105  0.955199
       depthK2            0.987596  0.997134
       uniform5           1.000000  1.000000
small  depthK             0.972740  0.978650
       depthK2            1.021866  1.008027
       uniform5           1.000000  1.000000

# 2. Std variances (per instance)

In [44]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','sampling_strategy','instance']).std() / df_gcnns.groupby(['type','sampling_strategy','instance']).mean()

In [45]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','sampling_strategy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   sampling_strategy                    
big    depthK             0.200893  0.175737
       depthK2            0.125737  0.112568
       uniform5           0.143056  0.131087
medium depthK             0.186327  0.110308
       depthK2            0.243768  0.153380
       uniform5           0.216847  0.130324
small  depthK             0.112221  0.067310
       depthK2            0.154374  0.086154
       uniform5           0.134978  0.068028

# 做一下 T-Test

# 计算Wins

In [46]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'uniform5'),metric]
        for sampling_strategy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,sampling_strategy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,sampling_strategy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,sampling_strategy),(metric,)] = [t_stats, p_value]

In [47]:
ttest_res

nnodes                  stime             
                           t_stats      p_value   t_stats      p_value
type   sampling_strategy                                              
big    depthK             -2.86041   0.00468481  -4.41191  1.68116e-05
       depthK2           -0.114493     0.908963 -0.448717     0.654126
       uniform5                  0            1         0            1
medium depthK             -3.57795  0.000435401  -4.54473  9.55857e-06
       depthK2           -0.226811     0.820805 -0.095125     0.924312
       uniform5                  0            1         0            1
small  depthK             -3.02385   0.00282624  -2.64514   0.00882103
       depthK2              1.6395     0.102697    1.0774     0.282611
       uniform5                  0            1         0            1

In [48]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type   instance                                                  seed
big    data/instances/cauctions/transfer_300_1500/instance_1.lp  21      600.0
                                                                 22      611.0
                                                                 23      607.0
                                                                 24      608.0
                                                                 25      609.0
                                                                         ...  
small  data/instances/cauctions/transfer_100_500/instance_9.lp   21      125.0
                                                                 22      126.0
                                                                 23      127.0
                                                                 24      123.0
                                                                 25      129.0
Name: stime, Length: 300, dtype: float64

In [49]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [50]:
len(winner_idx)

300

In [51]:
dfgcnns_wins = df_gcnns.groupby(['type', 'sampling_strategy'])['wins'].sum()
dfgcnns_wins

type    sampling_strategy
big     depthK               62
        depthK2              20
        uniform5             18
medium  depthK               50
        depthK2              31
        uniform5             19
small   depthK               41
        depthK2              25
        uniform5             34
Name: wins, dtype: int64

In [52]:
stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

Ttest_indResult(statistic=-2.455047672770633, pvalue=0.014949535302005438)

# 把Means、std 和 Wins合并起来

In [53]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std           wins
                            nnodes     stime    nnodes     stime wins
type   sampling_strategy                                             
big    depthK             0.953440  0.937425  0.200893  0.175737   62
       depthK2            1.003508  1.004854  0.125737  0.112568   20
       uniform5           1.000000  1.000000  0.143056  0.131087   18
medium depthK             0.937105  0.955199  0.186327  0.110308   50
       depthK2            0.987596  0.997134  0.243768  0.153380   31
       uniform5           1.000000  1.000000  0.216847  0.130324   19
small  depthK             0.972740  0.978650  0.112221  0.067310   41
       depthK2            1.021866  1.008027  0.154374  0.086154   25
       uniform5           1.000000  1.000000  0.134978  0.068028   34

In [60]:
# output_idx = pd.MultiIndex.from_product((dfgcnns_gmean_norm.index.get_level_values('type').unique(), sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx = pd.MultiIndex.from_product((['small', 'medium', 'big'], sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx

MultiIndex([( 'small', 'uniform5'),
            ( 'small',   'depthK'),
            ( 'small',  'depthK2'),
            ('medium', 'uniform5'),
            ('medium',   'depthK'),
            ('medium',  'depthK2'),
            (   'big', 'uniform5'),
            (   'big',   'depthK'),
            (   'big',  'depthK2')],
           names=['type', 'sampling_strategy'])

In [61]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [62]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   sampling_strategy                                  
small  uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN
medium uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN
big    uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN

In [63]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [64]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [65]:
out_df1

nnodes             stime  wins  \
type   sampling_strategy                                             
small  uniform5           1.0000r ± 13.50%   1.0000r ± 6.80%    34   
       depthK             0.9727r ± 11.22%   0.9787r ± 6.73%    41   
       depthK2            1.0219r ± 15.44%   1.0080r ± 8.62%    25   
medium uniform5           1.0000r ± 21.68%  1.0000r ± 13.03%    19   
       depthK             0.9371r ± 18.63%  0.9552r ± 11.03%    50   
       depthK2            0.9876r ± 24.38%  0.9971r ± 15.34%    31   
big    uniform5           1.0000r ± 14.31%  1.0000r ± 13.11%    18   
       depthK             0.9534r ± 20.09%  0.9374r ± 17.57%    62   
       depthK2            1.0035r ± 12.57%  1.0049r ± 11.26%    20   

                         t-stats (p-val)  
type   sampling_strategy                  
small  uniform5             0.00(1.0000)  
       depthK              -2.65(0.0088)  
       depthK2              1.08(0.2826)  
medium uniform5             0.00(1.0000)  
       depthK              -4.54(0.0000)  
       depthK2             -0.10(0.9243)  
big    uniform5             0.00(1.0000)  
       depthK              -4.41(0.0000)  
       depthK2             -0.45(0.6541)